In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import chart_studio.plotly as py # for the world map
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# EDA: World Happiness Report

In [ ]:
wh = pd.read_csv("whr_2017.csv") 
wh.describe()

In [ ]:
wh.head()

In [ ]:
# create a subset of the data
wh1 = wh[['Happiness.Score','Economy..GDP.per.Capita.','Family','Health..Life.Expectancy.', 'Freedom', 
          'Generosity','Trust..Government.Corruption.','Dystopia.Residual']]
cor = wh1.corr() # calculate the correlation of the feature matrix
sns.heatmap(cor, square = True);plt.show() # plot correlation as heat map

In [ ]:
# ACHTUNG: dieser Abschnitt ist nicht klausurrelevant!
# Reference: https://plot.ly/python/choropleth-maps/
data = dict(type = 'choropleth', 
           locations = wh['Country'],
           locationmode = 'country names',
           z = wh['Happiness.Score'], 
           text = wh['Country'],
           colorbar = {'title':'Happiness'})

layout = dict(title = 'Happiness Index 2017', 
             geo = dict(showframe = False, 
                       projection = {'type': 'mercator'}))

choromap3 = go.Figure(data = [data], layout=layout)
iplot(choromap3)

In [ ]:
wh1.head()

# StandardScaling (normalize to improve usage of Euclidian Distance)

In [ ]:
scaler = StandardScaler()
wh1_scaled = pd.DataFrame(scaler.fit_transform(wh1))

In [ ]:
wh1_scaled.head()

In [ ]:
sns.pairplot(data=wh1_scaled);plt.show()

# Perform Clustering

In [ ]:
model = KMeans(3)

model.fit(wh1_scaled)
clust_labels = model.predict(wh1_scaled)
clust_labels

# Inspect Results

In [ ]:
wh1_viz = wh1.copy()
wh1_viz['cluster'] = clust_labels

In [ ]:
sns.scatterplot(data=wh1_viz, x='Happiness.Score', y='Economy..GDP.per.Capita.', hue='cluster'); plt.show()

In [ ]:
sns.pairplot(data=wh1_viz, hue='cluster');plt.show()

In [ ]:
wh1_viz['Country'] = wh.iloc[:,0]

# nicht klausurrelevant
data = [dict(type='choropleth',
             locations = wh1_viz['Country'],
             locationmode = 'country names',
             z = wh1_viz['cluster'],
             text = wh1_viz['Country'],
             colorbar = {'title':'Cluster'})]

layout = dict(title='Clustering of Countries based on K-Means',
              geo=dict(showframe = False,
                       projection = {'type':'mercator'}))

map1 = go.Figure(data = data, layout=layout)
iplot(map1)

# Finding the optimal k
## Sum of squared errors for k=2

In [ ]:
# our clustering with k=2 leads to the following SSE
model.inertia_

# Optimizing k based on SSE (elbow method)

In [ ]:
k_range = range(1, 20)
cluster_SSEs = []
for num_clusters in k_range:
    clusters = KMeans(num_clusters)
    clusters.fit(wh1_scaled)
    cluster_SSEs.append(clusters.inertia_)

In [ ]:
# prepare visualization
cluster_SSEs_viz = pd.DataFrame(cluster_SSEs)
cluster_SSEs_viz['k'] = k_range
cluster_SSEs_viz.set_index('k', inplace=True)    
cluster_SSEs_viz.head(10)

In [ ]:
# nicht klausurrelevant (Erstellung des Plots!)
cluster_SSEs_viz.plot(marker = "o"); plt.show()
cluster_SSEs_viz.head(10)

The SSE is reduced significantly until k=7 --> the optimal k will probably range from 4-7

In [ ]:
model_opt = KMeans(5)

model_opt.fit(wh1_scaled)
clust_labels_opt = model_opt.predict(wh1_scaled)
clust_labels_opt

In [ ]:
wh1_viz_opt = wh1.copy()
wh1_viz_opt['cluster'] = clust_labels_opt
wh1_viz_opt['Country'] = wh.iloc[:,0]

data = [dict(type='choropleth',
             locations = wh1_viz_opt['Country'],
             locationmode = 'country names',
             z = wh1_viz_opt['cluster'],
             text = wh1_viz_opt['Country'],
             colorbar = {'title':'Cluster'})]

layout = dict(title='Clustering of Countries based on K-Means',
              geo=dict(showframe = False,
                       projection = {'type':'mercator'}))

map1 = go.Figure(data = data, layout=layout)
iplot(map1)

# Optimizing k based on a silhouette analysis

In [ ]:
# calculate the silhouette_score per k
# for silhouette see https://en.wikipedia.org/wiki/Silhouette_(clustering)
# ACHTUNG: genaue Berechnungsformel nicht klausurrelevant. Aber relevant: 
from sklearn.metrics import silhouette_score

k_range = range(2, 20)
cluster_scs = []
cluster_dbs = []
for num_clusters in k_range:
    model = KMeans(num_clusters)
    model.fit(wh1_scaled)
    clust_labels = model.predict(wh1_scaled)
    cluster_scs.append(silhouette_score(wh1, clust_labels))
    
print(cluster_scs)
pd.DataFrame(cluster_scs).plot(); plt.show()

In [ ]:
# visualizing the silhouette graphs
# see https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py

# ACHTUNG: nicht klausurrelevant, diesen Graph zu erstellen
# Aber: Interpretation relevant
# The silhouette plot displays a measure of how close each point in one cluster is to points in the neighboring 
# clusters and thus provides a way to assess parameters like number of clusters visually. 
# This measure has a range of [-1, 1].
# Silhouette coefficients (as these values are referred to as) near +1 indicate that the sample is far away from
# the neighboring clusters. A value of 0 indicates that the sample is on or very close to the decision boundary
# between two neighboring clusters and negative values indicate that those samples might have been assigned 
# to the wrong cluster.

from sklearn.metrics import silhouette_samples
import matplotlib.cm as cm
%matplotlib inline
X = wh1_scaled.copy()
range_n_clusters = range(2, 8)


for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(X.iloc[:, 0], X.iloc[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                c=colors, edgecolor='k')

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
                c="white", alpha=1, s=200, edgecolor='k')

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                    s=50, edgecolor='k')

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')

plt.show()